# Toronto Neighborhoods - Wikipedia Scraping and Clustering (Part 1 - 3) 
## !! Please note that all 3 parts are in the same Notebook!!

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests
from requests import get

#!conda install -c conda-forge beautifulsoup4 --yes # Installs beautifulsoup4
from bs4 import BeautifulSoup
#!pip install lxml

# Install libraries for lat/long lookup
#!pip install geopandas
#!pip install geopy

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # Install Folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Part 1 - Wikipedia Page Scraping

Defining dataframe and list for web scraping

In [2]:
column_names=['PostalCode','Borough','Neighborhood']
TorNhoods=pd.DataFrame(columns=column_names)

TNdata=[]

Using requests to read Wikipedia page (using an older version of the page for scraping)

In [3]:

torNpage=requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.',timeout=20).text

Use beautifulsoup to extract Postal Code data into a list

In [4]:
TNsoup = BeautifulSoup(torNpage,'lxml')

TNtable = TNsoup.find('table',class_='wikitable sortable')

for tr in TNtable.find_all('tr'):
        TDtable = (tr.find_all('td'))
        TNdata.append([index.text.strip() for index in TDtable]) 

Adding list to DataFrame and aggregating using GroupBy to build Toronto Neighborhoods data

In [5]:
TorNhoods = pd.DataFrame(TNdata,columns=column_names)
TorNhoods = TorNhoods.drop([0])

TorNhoods = TorNhoods[TorNhoods.Borough != 'Not assigned']
TorNhoods = TorNhoods.groupby(['PostalCode','Borough'],sort=False).agg(','.join)
TorNhoods = TorNhoods.reset_index()

TorNhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


Displaying Shape as per assignment requirement

In [6]:
TorNhoods.shape

(103, 3)

### Part 2 - Adding Latitude & Longitude to DataFrame using Geocoders
#### !! Please note that I tried using Nominatim, but results were poor, so used the location csv file


In [7]:
# This code was for trying to use Nominatim (OSM) to get Latitude/Longitude data but less than 30% of data was returned - hence abandoned

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderServiceError

locator = Nominatim(user_agent="arxGeocoder")
geolocator = RateLimiter(locator.geocode, min_delay_seconds=2, max_retries=0)

limit=0
success=0
failed=0

for index, row in TorNhoods.iterrows():
    limit = limit + 1
    print("Count = ",limit,"Post Code = ",row['PostalCode'])
    try:
        location = geolocator('{}, Toronto, Ontario'.format(row['PostalCode']),timeout=None)
        print(row['PostalCode'],location.latitude," ",location.longitude)
        success = success + 1
    except GeocoderServiceError as e:
        print(e)
    except:
        print("Nominatim Error - Unknown")
        failed = failed + 1
    if limit == 10:
        break

print("Query completed! Queried: ",limit,"Received: ",success," Failed:",failed)

Count =  1 Post Code =  M3A
M3A 43.6534817   -79.3839347
Count =  2 Post Code =  M4A
Nominatim Error - Unknown
Count =  3 Post Code =  M5A
Nominatim Error - Unknown
Count =  4 Post Code =  M6A
Nominatim Error - Unknown
Count =  5 Post Code =  M7A
M7A 43.6534817   -79.3839347
Count =  6 Post Code =  M9A
Nominatim Error - Unknown
Count =  7 Post Code =  M1B
M1B 43.6534817   -79.3839347
Count =  8 Post Code =  M3B
Nominatim Error - Unknown
Count =  9 Post Code =  M4B
Nominatim Error - Unknown
Count =  10 Post Code =  M5B
Nominatim Error - Unknown
Query completed! Queried:  10 Received:  3  Failed: 7


In [8]:
# Using supplied csv file to add Lat/Long to Toronto Neighborhood DataFrame

GeoData = pd.read_csv("http://cocl.us/Geospatial_data")
GeoData.rename(columns={'Postal Code':'PostalCode'},inplace=True)
GeoData = GeoData.set_index('PostalCode')

TorNhoods = TorNhoods.set_index('PostalCode')
tnh_enrich=pd.merge(TorNhoods,GeoData,left_index=True, right_index=True)

# Selecting only Boroughs with Toronto in name to limit to downtown/city locations
toronto_only = tnh_enrich[tnh_enrich['Borough'].str.contains('Toronto')].reset_index(drop=True)

### Part 3 - Segmenting and Clustering (only Boroughs with Toronto in name)
#### Replicates the exercise - Segmenting and Clustering Neighborhoods in New York City

In [9]:
# Getting Toronto coordinates from Nominatim
yyzloc = geolocator('Toronto City, Ontario, Canada',timeout=None)

# Generating map of Toronto using folium
map_toronto = folium.Map(location=[yyzloc.latitude, yyzloc.longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_only['Latitude'], toronto_only['Longitude'], toronto_only['Borough'], toronto_only['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
#Setting up Foursquare credentials
CLIENT_ID = '33SCCYRTKM2435UIGC52SX2NKT5ELO0E5QVIECBCRIQE2IJB' # your Foursquare ID
CLIENT_SECRET = '0QZEB51NYB0KQLXUAXTUXFSPCH4GOSK40PSSYRAXW0PLB1ZU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 33SCCYRTKM2435UIGC52SX2NKT5ELO0E5QVIECBCRIQE2IJB
CLIENT_SECRET:0QZEB51NYB0KQLXUAXTUXFSPCH4GOSK40PSSYRAXW0PLB1ZU


In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to repeat the same process to all the neighborhoods with Toronto in its name

In [12]:
# Defining a function to gather nearby venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called toronto_venues

In [13]:
# Calling function for all values in Toronto DataFrame

LIMIT = 100

toronto_venues = getNearbyVenues(names=toronto_only['Neighborhood'],
                                   latitudes=toronto_only['Latitude'],
                                   longitudes=toronto_only['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(1688, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Check how many venues were returned for each neighborhood

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,78,78,78,78,78,78
"Chinatown,Grange Park,Kensington Market",81,81,81,81,81,81
Christie,17,17,17,17,17,17


Check how many unique categories are returned

In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


Analyzing each Neighborhood

In [17]:
# One hot encoding for Toronto DataFrame

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
toronto_onehot.shape

(1688, 233)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
# Grouping by Neighborhoods

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.000000,0.00000,0.00,0.012821,0.000000,0.000000,0.012821,0.000000,0.000000,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.049383,0.000000,0.061728,0.012346,0.000000,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.024096,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.012048,0.012048,0.00


Print each neighborhood along with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1       Restaurant  0.06
2             Café  0.05
3           Bakery  0.03
4  Thai Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3         Cheese Shop  0.04
4          Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2       Nightclub  0.08
3     Coffee Shop  0.08
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.06
2                   Spa  0.06
3  Gym / Fitness Center  0.06
4         Garden Center  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  

Function to sort the venues in descending order

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Café,Bar,Bakery,Thai Restaurant,Salad Place,Lounge,Cosmetics Shop,Steakhouse
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Japanese Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Café,Breakfast Spot,Nightclub,Coffee Shop,Grocery Store,Gym,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Skate Park,Smoke Shop,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Coffee Shop,Plane,Harbor / Marina


#### Cluster Neighborhoods using K-Means into 5 clusters

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_only

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Café,Bakery,Theater,Spa
1,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Burrito Place,Indian Restaurant,Beer Bar,Italian Restaurant,Nightclub,Seafood Restaurant,Juice Bar,Sandwich Place
2,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Restaurant,Ramen Restaurant,Lingerie Store
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Clothing Store,Cosmetics Shop,Bakery,Breakfast Spot,Diner
4,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Visualizing the resulting clusters using Folium

In [25]:
# create map
map_clusters = folium.Map(location=[yyzloc.latitude, yyzloc.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters

Cluster 0 - Seems to have a high density of Coffee Shops, Cafes and Restaurants as most common venues

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,Coffee Shop,Park,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Café,Bakery,Theater,Spa
1,Queen's Park,Coffee Shop,Yoga Studio,Burrito Place,Indian Restaurant,Beer Bar,Italian Restaurant,Nightclub,Seafood Restaurant,Juice Bar,Sandwich Place
2,"Ryerson,Garden District",Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Restaurant,Ramen Restaurant,Lingerie Store
3,St. James Town,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Clothing Store,Cosmetics Shop,Bakery,Breakfast Spot,Diner
4,The Beaches,Health Food Store,Trail,Pub,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Japanese Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Thai Restaurant,Japanese Restaurant,Burger Joint,Department Store,Café,Bubble Tea Shop,Salad Place
7,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Baby Store,Candy Store,Gas Station
8,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Café,Bar,Bakery,Thai Restaurant,Salad Place,Lounge,Cosmetics Shop,Steakhouse
9,"Dovercourt Village,Dufferin",Pharmacy,Bakery,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Coffee Shop,Pool,Café,Brewery


Cluster 1 - Seems to have a high density of Playground and Stores as most common venues

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park,Summerhill East",Playground,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 2 - Seems to have a high density of Garden and Stores as most common venues

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Garden,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 3 - Seems to have a high density of Parks, Playground, Trails as most common venues

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North,Forest Hill West",Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
33,Rosedale,Park,Playground,Trail,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 4 - Seems to have a high density of Park and Swim Schools as most common venues

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Swim School,Bus Line,Women's Store,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
